1. Buatlah sebuah Sistem REST API dengan menggunakan dua data di atas
2. Buatlah 1 tabel tambahan yang mendukung backend aplikasi kalian berdasar pada dua berkas di atas. (Silahkan masukkan 3 tabel tersebut dalam MySQL lokal dan berkreasi dengannya)
3. Beberapa berkas yang harus ada
a. models
b. router
4. Untuk berkas lain jika dirasa dibutuhkan silahkan tambahkan, misal schema, berkas utama (baik diberi nama main atau app), dll (konfigurasi .env sesuaikan dengan materi)
5. Setiap Tabel harus ada relasi dengan minimal satu tabel lainnya.
6. Gunakan ORM
7. Silahkan memilih menggunakan Flask atau FastAPI
8. Ada beberapa endpoint api yang harus ada
9. Bisa melihat semua barang
10. Bisa melihat barang dengan kata kunci (jika dicari buku, maka semua barang yang ada kata "buku" akan masuk pencarian)
11. Silahkan tambahkan endpoint lain sesuai kebutuhan
12. Aplikasikan penggunaan JWT, contohnya user hanya bisa melihat data transaksi user tersebut tapi tidak user lain (hanya contoh). product tidak perlu JWT (karena semua user bisa melihat semua product)
13. Dokumetasikan API kalian dengan Swagger
14. Semua tambahan dari ketentuan di atas akan menjadi nilai tambah
15. Jelaskan tentang aplikasi backend kalian pada readme.md

In [1]:
import pandas as pd

df_users = pd.read_csv('users.csv')

display(df_users)

,customer_id,name,city,state,postal
0,1,Claire Gute,Fremont,Nebraska,68025
1,2,Sean O'Donnell,Provo,Utah,84604
2,3,Brosina Hoffman,Keller,Texas,76248
3,4,Andrew Allen,Harlingen,Texas,78550
4,5,Irene Maddox,Kenner,Louisiana,70065
...,...,...,...,...,...
404,405,Ionia McGrath,Plainfield,New Jersey,7060
405,406,Anemone Ratner,Mission Viejo,California,92691
406,407,Sung Chung,Abilene,Texas,79605
407,408,Ricardo Emerson,Jackson,Michigan,49201


In [2]:
df_products = pd.read_csv('products.csv', sep=';', encoding='latin-1')
df_products.columns = ['product_id', 'product_name', 'category', 'sub_category']
display(df_products)

,product_id,product_name,category,sub_category
0,1,Bush Somerset Collection Bookcase,Furniture,Bookcases
1,2,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture,Chairs
2,3,Self-Adhesive Address Labels for Typewriters b...,Office Supplies,Labels
3,4,Bretford CR4500 Series Slim Rectangular Table,Furniture,Tables
4,5,Eldon Fold 'N Roll Cart System,Office Supplies,Storage
...,...,...,...,...
1845,1846,RCA ViSYS 25425RE1 Corded phone,Technology,Phones
1846,1847,Cisco 8961 IP Phone Charcoal,Technology,Machines
1847,1848,"Eureka Hand Vacuum, Bagless",Office Supplies,Appliances
1848,1849,LG G2,Technology,Phones


In [3]:
import random
from datetime import datetime, timedelta

random.seed(7)

def random_tanggal():
    # Set the start and end dates
    start_date = datetime(2020, 1, 1)
    end_date = datetime(2022, 12, 17)
    # Generate a random date between the start and end dates
    random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    # Convert the random date to a string in the DD--MM-YY format
    formatted_date = random_date.strftime("%d-%m-20%y")
    return formatted_date

n = 1000 #banyaknya kolom

df_purchase = pd.DataFrame({'date':[random_tanggal() for i in range(n)],
                         'name':[random.choices(df_users['name'])[0] for i in range(n)], 
                         'product_name':[random.choices(df_products['product_name'])[0] for i in range(n)],
                         'quantity':[random.randint(1,25) for i in range(n)]
                        })
display(df_purchase)

,date,name,product_name,quantity
0,25-10-2021,Matt Collins,Bevis 44 x 96 Conference Tables,14
1,04-11-2020,Lena Cacioppo,HTC One Mini,8
2,19-03-2022,Max Jones,Acco Suede Grain Vinyl Round Ring Binder,8
3,08-04-2020,Dan Lawera,Global Deluxe High-Back Office Chair in Storm,12
4,28-05-2020,Liz Preis,"Avery Trapezoid Ring Binder, 3"" Capacity, Blac...",7
...,...,...,...,...
995,01-10-2021,Carl Ludwig,Boston 16765 Mini Stand Up Battery Pencil Shar...,2
996,08-06-2020,Joy Daniels,"IBM Multi-Purpose Copy Paper, 8 1/2 x 11"", Case",22
997,20-02-2021,Ralph Ritter,"Fellowes Premier Superior Surge Suppressor, 10...",4
998,05-03-2020,Shui Tom,Martin Yale Chadless Opener Electric Letter Op...,18


In [4]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from dotenv import dotenv_values
import urllib.parse

params = {'db':dotenv_values('TugasAkhir.env')}

db_user = params['db']['MYSQL_USERNAME']
db_password = urllib.parse.quote_plus(params['db']['MYSQL_PASSWORD'])
db_host = params['db']['MYSQL_HOST']
db_port = params['db']['MYSQL_PORT']
db_name = params['db']['MYSQL_DB']

# Buat koneksi ke database MySQL
SQLALCHEMY_DATABASE_URL = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(SQLALCHEMY_DATABASE_URL, echo=True, pool_pre_ping=True)

# Buat base class untuk ORM
Base = declarative_base()

try:
    # Buat tabel products
    class Products(Base):
        __tablename__ = 'products'
        product_id = Column(Integer, nullable=False)
        product_name = Column(String(50), primary_key=True)
        category = Column(String(50), nullable=False)
        sub_category = Column(String(50), nullable=False)
    
    # Buat tabel users
    class Users(Base):
        __tablename__ = 'users'
        customer_id = Column(Integer, nullable=False)
        name = Column(String(50), primary_key=True)
        city = Column(String(50), nullable=False)
        state = Column(String(50), nullable=False)
        postal = Column(Integer, nullable=False)

    # Buat tabel purchase
    class Purchase(Base):
        __tablename__ = 'purchase'
        name = Column(String(50), nullable=False, primary_key=True)
        date = Column(DateTime, nullable=False)
        product_name = Column(String(50), nullable=False, primary_key=True)
        quantity = Column(Integer, nullable=False)
    
    Base.metadata.create_all(engine)
except Exception as e:
    print(e)    

2022-12-23 04:16:29,024 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-12-23 04:16:29,026 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 04:16:29,029 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-12-23 04:16:29,029 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 04:16:29,031 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-12-23 04:16:29,032 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 04:16:29,034 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-23 04:16:29,035 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 04:16:29,036 INFO sqlalchemy.engine.Engine [generated in 0.00100s] {'table_schema': 'penjualan_uzu', 'table_name': 'products'}
2022-12-23 04:16:29,040 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 04:16:2

In [5]:
# Insert data ke tabel products
df_products.to_sql('products', engine, if_exists='replace', index=False)

# Insert data ke tabel users
df_users.to_sql('users', engine, if_exists='replace', index=False)

# Insert data ke tabel purchase
df_purchase.to_sql('purchase', engine, if_exists='replace', index=False)

2022-12-23 04:16:29,059 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 04:16:29,060 INFO sqlalchemy.engine.Engine [cached since 0.02516s ago] {'table_schema': 'penjualan_uzu', 'table_name': 'products'}
2022-12-23 04:16:29,064 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 04:16:29,065 INFO sqlalchemy.engine.Engine [cached since 0.02966s ago] {'table_schema': 'penjualan_uzu', 'table_name': 'products'}
2022-12-23 04:16:29,068 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `penjualan_uzu`
2022-12-23 04:16:29,069 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 04:16:29,088 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `products`
2022-12-23 04:16:29,089 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 04:16:29,094 INFO sqlalchemy.engine.Engine BEGIN (impl

-1

In [6]:
# Buat Session
Session = sessionmaker(bind=engine)
session = Session()
conn = engine.connect()

In [7]:
conn.execute('select * from users').all()

2022-12-23 04:16:29,880 INFO sqlalchemy.engine.Engine select * from users
2022-12-23 04:16:29,881 INFO sqlalchemy.engine.Engine [raw sql] {}


[(1, 'Claire Gute', 'Fremont', 'Nebraska', 68025),
 (2, "Sean O'Donnell", 'Provo', 'Utah', 84604),
 (3, 'Brosina Hoffman', 'Keller', 'Texas', 76248),
 (4, 'Andrew Allen', 'Harlingen', 'Texas', 78550),
 (5, 'Irene Maddox', 'Kenner', 'Louisiana', 70065),
 (6, 'Pete Kriz', 'Santa Ana', 'California', 92704),
 (7, 'Alejandro Grove', 'Pharr', 'Texas', 78577),
 (8, 'Zuschuss Donatelli', 'Taylor', 'Michigan', 48180),
 (9, 'Sandra Flanagan', 'New Rochelle', 'New York', 10801),
 (10, 'Emily Burns', 'Oswego', 'Illinois', 60543),
 (11, 'Eric Hoffmann', 'Santa Barbara', 'California', 93101),
 (12, 'Tracy Blumstein', 'Lorain', 'Ohio', 44052),
 (13, "Patrick O'Donnell", 'Bloomington', 'Illinois', 61701),
 (14, 'Lena Hernandez', 'Atlanta', 'Georgia', 30318),
 (15, 'Darren Powers', 'Moreno Valley', 'California', 92553),
 (16, 'Ted Butterfield', 'Redlands', 'California', 92374),
 (17, 'Kunst Miller', 'Philadelphia', 'Pennsylvania', 19140),
 (18, 'Karen Daniels', 'Park Ridge', 'Illinois', 60068),
 (19, '

In [ ]:
import pandas as pd
import random
random.seed(7)
from datetime import datetime, timedelta
import urllib.parse
from dotenv import dotenv_values

import sqlalchemy
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base

from fastapi import FastAPI, File, UploadFile
import uvicorn
import nest_asyncio
nest_asyncio.apply()

params = {'db':dotenv_values('TugasAkhir.env')}

db_user = params['db']['MYSQL_USERNAME']
db_password = urllib.parse.quote_plus(params['db']['MYSQL_PASSWORD'])
db_host = params['db']['MYSQL_HOST']
db_port = params['db']['MYSQL_PORT']
db_name = params['db']['MYSQL_DB']

# Buat koneksi ke database MySQL
SQLALCHEMY_DATABASE_URL = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(SQLALCHEMY_DATABASE_URL, echo=True, pool_pre_ping=True)

# Membuat tanggal acak
def random_tanggal():
    # Set the start and end dates
    start_date = datetime(2020, 1, 1)
    end_date = datetime(2022, 12, 17)
    # Generate a random date between the start and end dates
    random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    # Convert the random date to a string in the DD--MM-YYYY format
    formatted_date = random_date.strftime("%d-%m-20%y")
    return formatted_date

# Buat base class untuk ORM
Base = declarative_base()

try:
    # Buat tabel products
    class Products(Base):
        __tablename__ = 'products'
        product_id = Column(Integer, nullable=False)
        product_name = Column(String(50), primary_key=True)
        category = Column(String(50), nullable=False)
        sub_category = Column(String(50), nullable=False)

    # Buat tabel users
    class Users(Base):
        __tablename__ = 'users'
        customer_id = Column(Integer, nullable=False)
        name = Column(String(50), primary_key=True)
        city = Column(String(50), nullable=False)
        state = Column(String(50), nullable=False)
        postal = Column(Integer, nullable=False)

    # Buat tabel purchase
    class Purchase(Base):
        __tablename__ = 'purchase'
        name = Column(String(50), nullable=False, primary_key=True)
        date = Column(DateTime, nullable=False)
        product_name = Column(String(50), nullable=False, primary_key=True)
        quantity = Column(Integer, nullable=False)

    Base.metadata.create_all(engine)
except Exception as e:
    print(e)        

app = FastAPI()

def tabel_sql_jadi(users_csv,products_csv):
    df_users = pd.read_csv(users_csv.file)
    df_products = pd.read_csv(products_csv.file, sep=';', encoding='latin-1')
    df_products.columns = ['product_id', 'product_name', 'category', 'sub_category']

    n = 1000 #banyaknya kolom tabel purchase

    df_purchase = pd.DataFrame({'date':[random_tanggal() for i in range(n)],
                         'name':[random.choices(df_users['name'])[0] for i in range(n)], 
                         'product_name':[random.choices(df_products['product_name'])[0] for i in range(n)],
                         'quantity':[random.randint(1,25) for i in range(n)]
                        })
    
    # Insert data ke tabel products
    df_products.to_sql('products', engine, if_exists='replace', index=False)

    # Insert data ke tabel users
    df_users.to_sql('users', engine, if_exists='replace', index=False)

    # Insert data ke tabel purchase
    df_purchase.to_sql('purchase', engine, if_exists='replace', index=False)
    
# Buat Session
Session = sessionmaker(bind=engine)
session = Session()
conn = engine.connect()

# Menampilkan isi dari tabel (users, products, purchase)
@app.post("/tabel/{nama_tabel}")
async def tampilkan_isi_tabel(users_csvfile: UploadFile = File(...), 
                          products_csvfile: UploadFile = File(...),
                          nama_tabel = None
                         ):
    tabel_sql_jadi(users_csvfile,products_csvfile)
    
    return conn.execute(f'select * from {nama_tabel.lower()}').all()
        
if __name__ == "__main__":
    uvicorn.run(app)
    
    
    

2022-12-23 04:25:26,621 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-12-23 04:25:26,623 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 04:25:26,625 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-12-23 04:25:26,626 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 04:25:26,628 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-12-23 04:25:26,629 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 04:25:26,631 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-23 04:25:26,633 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 04:25:26,634 INFO sqlalchemy.engine.Engine [generated in 0.00114s] {'table_schema': 'penjualan_uzu', 'table_name': 'products'}
2022-12-23 04:25:26,637 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 04:25:2

INFO:     Started server process [21500]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:61636 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:61636 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:61636 - "POST /ajax.php?v=1671744332617 HTTP/1.1" 404 Not Found
2022-12-23 04:25:52,883 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 04:25:52,884 INFO sqlalchemy.engine.Engine [cached since 26.25s ago] {'table_schema': 'penjualan_uzu', 'table_name': 'products'}
2022-12-23 04:25:52,888 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 04:25:52,889 INFO sqlalchemy.engine.Engine [cached since 26.26s ago] {'table_schema': 'penjualan_uzu', 'table_name': 'products'}
2022-12-23 04:25:52,894 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `penjualan_uzu`
2022-12-23 04:25:52,895 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 04:2